In [28]:
import os
import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)

insert_amd_env_vars =  int(configParser.get('COMMON', 'insert_amd_env_vars'))
HSA_OVERRIDE_GFX_VERSION =  configParser.get('COMMON', 'HSA_OVERRIDE_GFX_VERSION')
ROCM_PATH =  configParser.get('COMMON', 'ROCM_PATH')

if(insert_amd_env_vars != 0):
    os.environ["HSA_OVERRIDE_GFX_VERSION"] = HSA_OVERRIDE_GFX_VERSION
    os.environ["ROCM_PATH"] = ROCM_PATH
    
#import os
#os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
#os.environ["AUTOGRAPH_VERBOSITY"] = "0"

In [29]:

import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)


video_path =  configParser.get('test_imagen','video_path')

ttwbdf =  int(configParser.get('test_imagen', 'time_to_wait_before_deleting_files'))


cuda =  int(configParser.get('COMMON', 'cuda'))
cpus =  int(configParser.get('COMMON', 'cpus'))




In [30]:
#TODO document jupyter
import pickle
import sqlite3 as sl
import pandas as pd
import numpy as np
from PIL import Image

from textwrap import wrap
import os


import matplotlib.pyplot as plt
import numpy as np
import pandas as pd



In [31]:
pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None


In [32]:
audio_folder = output_folder = r'imagen_testing_folder' 
if not os.path.exists(audio_folder):
    os.makedirs(audio_folder)

In [33]:
image_size =  int(configParser.get('COMMON', 'resizeImageTo'))
audio_length_used =  configParser.get('test_imagen', 'audio_length_used') 
model_filename =  configParser.get('test_imagen', 'model_filename') + '_'  + audio_length_used +  's.pt'
openl3_mode =  configParser.get('test_imagen', 'openl3_mode')
folder =  configParser.get('test_imagen', 'folder')
number_of_images =  configParser.get('test_imagen', 'number_of_images')

In [34]:
age =  int(configParser.get('test_imagen', 'age'))
ethnicity =  str(configParser.get('test_imagen', 'ethnicity'))
gender =  str(configParser.get('test_imagen', 'gender'))
language =  str(configParser.get('test_imagen', 'language'))

In [35]:
dataGotten = [['1',video_path,age,ethnicity,gender,[],[],language,[]]]
df = pd.DataFrame(dataGotten,columns = ['ID','VIDEO_PATH','caption_a','caption_e','caption_g','SPEAKER_EMB','AUDIO_EMB','caption_l','image_path'])


In [36]:
#import tensorflow as tf
#gpus = tf.config.experimental.list_physical_devices('GPU')
#for gpu in gpus:
#  tf.config.experimental.set_memory_growth(gpu, True)

In [37]:
#gpus

In [38]:
from testing_imagen_speechbrain import extract_speechbrain_embeddings
from testing_imagen_pyannote_titanet import extract_pyannote_titanet_embeddings
import multiprocessing

audio_embs =  configParser.get('COMMON', 'audio_embs') 
from pydub import AudioSegment
import math
# Get original duration of video
audio = AudioSegment.from_file(video_path)
audio_length_og = math.floor(audio.duration_seconds)


video_filename = os.path.basename(video_path)

absPathAudio_w = os.path.abspath(audio_folder) + "/" + video_filename
absPathAudio = os.path.splitext(absPathAudio_w)[0]+'_audio.wav'
absPathAudio_w = os.path.splitext(absPathAudio_w)[0]


import subprocess
# Extract audio monochannel and with 16khz and put it in absPathAudio

command = "ffmpeg -nostats -loglevel 0 -y -i '" + video_path + "' -acodec pcm_s16le -ab 160k -ac 1 -ar 16000 -vn '" + absPathAudio  + "'"
subprocess.call(command, shell=True)

from threading import Thread
import time

# Function to delete audio temp files
def delFiles(filesToDelete):
    time.sleep(ttwbdf)  # wait a bit
    for file in filesToDelete:  
        try:
            os.remove(file)
        except OSError:
            pass


# Will either truncate or loop the original video to reach audio_length (6,12 or 24)
audio_length_list = [24]
for audio_length in audio_length_list:
    path_var_len_audio =  absPathAudio_w + "audio" + str(audio_length) + "s.wav"    # path to the variable length audio
    path_var_len_audio_temp =  absPathAudio_w + "audio_temp" + str(audio_length) + "s.wav"  # path to a temp version of the variable length audio

    if(audio_length_og > audio_length):
        # Truncate    

        command = "ffmpeg -nostats -loglevel 0 -y -ss 0 -t "+str(audio_length)+" -i \"" + absPathAudio + "\" \"" + path_var_len_audio + "\""
        subprocess.call(command, shell=True)


    else:
        # Loop then truncaate
        #print("lesa")
        twoDigitLenStr = f"{audio_length:02}"
        #print(twoDigitLenStr)
        command = "ffmpeg -nostats -loglevel 0 -y -stream_loop -1 -i '" + absPathAudio + "' -t \"00:00:"+twoDigitLenStr+".000\" -codec:a \"aac\" -f \"wav\" -c copy '"+ path_var_len_audio_temp + "'"
        subprocess.call(command, shell=True)
        command = "ffmpeg -nostats -loglevel 0 -y -ss 0 -t "+str(audio_length)+" -i \"" + path_var_len_audio_temp + "\" \"" + path_var_len_audio + "\""
        subprocess.call(command, shell=True)




    q = multiprocessing.Queue()

    proc = multiprocessing.Process(target=extract_speechbrain_embeddings, args=(q,path_var_len_audio,))
    proc.start()
    proc.join()



    df['SPEAKER_EMB'] = q.get()
    proc = multiprocessing.Process(target=extract_pyannote_titanet_embeddings, args=(q,path_var_len_audio,audio_embs,audio_length,openl3_mode,))
    proc.start()
    proc.join()

    df['AUDIO_EMB'] = q.get()

    

    ftd = [absPathAudio,path_var_len_audio,os.path.basename(path_var_len_audio),path_var_len_audio_temp]
    tDelete = Thread(target=delFiles, args=(ftd,))   # spawn a process
    tDelete.start()



Lightning automatically upgraded your loaded checkpoint from v1.2.7 to v1.9.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../.cache/torch/pyannote/models--pyannote--embedding/snapshots/20b2db779562a3141f5eadd34a0232dbcd56d620/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.8.1+cu102, yours is 2.0.1+cu117. Bad things might happen unless you revert torch to 1.x.


Lightning automatically upgraded your loaded checkpoint from v1.2.7 to v1.9.4. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../.cache/torch/pyannote/models--pyannote--embedding/snapshots/20b2db779562a3141f5eadd34a0232dbcd56d620/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.8.1+cu102, yours is 2.0.1+cu117. Bad things might happen unless you revert torch to 1.x.


[NeMo W 2023-07-09 10:15:28 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /manifests/combined_fisher_swbd_voxceleb12_librispeech/train.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    is_tarred: false
    tarred_audio_filepaths: null
    tarred_shard_strategy: scatter
    augmentor:
      noise:
        manifest_path: /manifests/noise/rir_noise_manifest.json
        prob: 0.5
        min_snr_db: 0
        max_snr_db: 15
      speed:
        prob: 0.5
        sr: 16000
        resample_type: kaiser_fast
        min_speed_rate: 0.95
        max_speed_rate: 1.05
    num_workers: 15
    pin_memory: true
    
[NeMo W 2023-07-09 10:15:28 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method 

[NeMo I 2023-07-09 10:15:28 features:287] PADDING: 16
[NeMo I 2023-07-09 10:15:29 save_restore_connector:249] Model EncDecSpeakerLabelModel was successfully restored from /home/gamal/.cache/huggingface/hub/models--nvidia--speakerverification_en_titanet_large/snapshots/4e0b2d387a805da7c208b13d5898ee09de8ec1e9/speakerverification_en_titanet_large.nemo.
[NeMo I 2023-07-09 10:15:29 cloud:58] Found existing object /home/gamal/.cache/torch/NeMo/NeMo_1.18.1/speakerverification_speakernet/a8330fa516557b963a89ccbf0fcbe2f2/speakerverification_speakernet.nemo.
[NeMo I 2023-07-09 10:15:29 cloud:64] Re-using file from: /home/gamal/.cache/torch/NeMo/NeMo_1.18.1/speakerverification_speakernet/a8330fa516557b963a89ccbf0fcbe2f2/speakerverification_speakernet.nemo
[NeMo I 2023-07-09 10:15:29 common:913] Instantiating model from pre-trained checkpoint


[NeMo W 2023-07-09 10:15:29 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /ws/manifests/raid/combined/train_manifest.json
    sample_rate: 16000
    labels: null
    batch_size: 64
    shuffle: true
    time_length: 8
    augmentor:
      noise:
        manifest_path: /ws/manifests/raid/musan/musan_music_noise_manifest_dur8.json
        prob: 0.2
        min_snr_db: 5
        max_snr_db: 15
    num_workers: 4
    
[NeMo W 2023-07-09 10:15:29 modelPT:168] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration file to setup the validation data loader(s). 
    Validation config : 
    manifest_filepath: /ws/manifests/raid/voxceleb/small_manifest.json
    sample_rate: 16000
    labels: null
    batch_size:

[NeMo I 2023-07-09 10:15:29 features:287] PADDING: 16
[NeMo I 2023-07-09 10:15:29 save_restore_connector:249] Model EncDecSpeakerLabelModel was successfully restored from /home/gamal/.cache/torch/NeMo/NeMo_1.18.1/speakerverification_speakernet/a8330fa516557b963a89ccbf0fcbe2f2/speakerverification_speakernet.nemo.


In [39]:
df3 = df[["image_path","caption_a","caption_e","caption_g","caption_l"]]
data_frame = df3
data_frame['SPEAKER_EMB'] = df['SPEAKER_EMB']
data_frame['AUDIO_EMB'] = df['AUDIO_EMB']

In [40]:
import pickle


with open(output_folder + '/' + 'filename.pickle', 'wb') as handle:
    pickle.dump(data_frame, handle, protocol=pickle.HIGHEST_PROTOCOL)